In [ ]:
!pip install retry

In [ ]:
import os
import requests
import shutil
from joblib import Parallel, delayed
from bs4 import BeautifulSoup
from retry import retry

FREE_FOLDER = "images"
PREMIUM_FOLDER = "images-premium"

os.makedirs(FREE_FOLDER, exist_ok=True)
os.makedirs(PREMIUM_FOLDER, exist_ok=True)

@retry()
def download_image(image_url, folder):
  filename = image_url.split("/")[-1]
  image_path = os.path.join(folder, filename)

  resp = requests.get(image_url, stream = True)
  resp.raise_for_status()
  resp.raw.decode_content = True

  with open(image_path, "wb") as img:
    shutil.copyfileobj(resp.raw, img)

@retry()
def extract_images(page_url):
  resp = requests.get(page_url)
  resp.raise_for_status()


  soup = BeautifulSoup(resp.text, features="lxml")
  images = soup.find_all("li", attrs={"class": "icon"})

  for image_block in images:
    if 'icon-fill' in image_block['class']:
      continue
    image_url = image_block.find("img")['data-src'].replace("png/128", "png/512")
    yield image_block['data-premium'] == '1', image_url


def process_page(page):
  page_url = "https://www.flaticon.com/free-icons/logo/{}".format(page)

  images = extract_images(page_url)
  for is_premium, image_url in images:
    folder = FREE_FOLDER if not is_premium else PREMIUM_FOLDER

    download_image(image_url, folder)

In [ ]:
from tqdm import trange

Parallel(n_jobs=16)(delayed(process_page)(page) for page in trange(1, 185))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
! ls -la images | wc -l
! ls -la images-premium | wc -l

13244
3606


In [ ]:
! zip -r images.zip images images-premium

Выходные данные были обрезаны до нескольких последних строк (5000).
  adding: images/915835.png (deflated 5%)
  adding: images/1245007.png (deflated 2%)
  adding: images/1272408.png (deflated 4%)
  adding: images/2103763.png (deflated 20%)
  adding: images/1349752.png (deflated 2%)
  adding: images/2702601.png (deflated 5%)
  adding: images/2111308.png (deflated 27%)
  adding: images/894179.png (deflated 6%)
  adding: images/882826.png (deflated 2%)
  adding: images/2554806.png (deflated 32%)
  adding: images/888900.png (deflated 14%)
  adding: images/2111534.png (deflated 2%)
  adding: images/207094.png (deflated 5%)
  adding: images/732246.png (deflated 8%)
  adding: images/1400842.png (deflated 3%)
  adding: images/408734.png (deflated 3%)
  adding: images/733625.png (deflated 5%)
  adding: images/356112.png (deflated 4%)
  adding: images/2620629.png (deflated 10%)
  adding: images/311349.png (deflated 6%)
  adding: images/376169.png (deflated 20%)
  adding: images/2506336.png (defl

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!cp images.zip /gdrive/My\ Drive/logos.zip

Mounted at /gdrive
